# Alternating Least Square (ALS)

https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1

In [1]:
import pandas as pd
import clicks_weigher, orders_weigher, InteractionTable

In [2]:
def get_clicks():
    path = '../data/clicks/click.pkl'
    return pd.read_pickle(path).head(100)

def get_orders():
    path = '../data/orders/orders.pkl'
    return pd.read_pickle(path).head(100)

In [3]:
interactions = InteractionTable(get_clicks,
                                get_orders,
                                clicks_weigher,
                                orders_weigher,
                                alpha=0.5)

Clicks df loaded: size=100,  uniq_users=49,  uniq_chains=83
Clicks weighter: use user clicks per chain as weight
Clicks df weighted: size=88, uniq_users=49, uniq_chains=83
Orders df loaded: size=100,  uniq_users=100,  uniq_chains=78
Orders weighter: use successful user orders per chain as weight
Orders df weighted: size=100, uniq_users=100, uniq_chains=78


In [4]:
from sklearn.decomposition import TruncatedSVD
from collections import defaultdict
from numpy import corrcoef, array

class TruncatedSVDModel:
    
    def __init__(self, interaction_table, n_components=5, n_iter=7, random_state=42):
        self.interaction_tab = interaction_table
        svd = TruncatedSVD(n_components=n_components, random_state=random_state, n_iter=n_iter)
        decomposed_mat = svd.fit_transform(self.interaction_tab.sparse_interaction_matrix)
        self.corr = corrcoef(decomposed_mat)
        
    def __get_most_correlated(self, pred, df, min_corr, max_corr, top_k):
        """
        returns dict: user_id -> list of predicted chains
        predicted chains are most highly correlated chains with user history (orders or clicks)
        """
        for user_id, predicted_top in pred.items():
            if len(predicted_top) >= top_k:
                continue
            user_chains = set(df.query('user_id == @user_id')['chain_id'])
            # TODO: add iterations logger
            for user_chain in user_chains:
                # get most highly correlated chains
                corr = self.corr[self.interaction_tab.chain_index[user_chain]]
                corr = array(sorted(corr, reverse=True))
                for i, in_range in enumerate((corr > min_corr) == (corr < max_corr)):
                    top_correlated_chain = self.interaction_tab.r_chain_index[i]
                    # add to answer if needed
                    if in_range and len(predicted_top) < top_k and top_correlated_chain not in user_chains:
                        predicted_top.append(top_correlated_chain)
        return pred
        
    def predict(self, user_ids, min_corr=0.7, max_corr=0.99, top_k=30):
        """
        returns dict: user_id -> list of predicted chains
        """
        pred = defaultdict(list)
        for user_id in user_ids:
            pred[user_id] = []
        # try to get predictions with most correlated items by orders history
        pred = self.__get_most_correlated(pred, self.interaction_tab.orders_df, min_corr, max_corr, top_k)
        # fill the rest by clicks history
        pred = self.__get_most_correlated(pred, self.interaction_tab.clicks_df, min_corr, max_corr, top_k)
        return pred

In [5]:
model = TruncatedSVDModel(interactions)
model.predict([2592128, 7371423], top_k=5)

defaultdict(list, {2592128: [32441, 32449, 32697, 32877, 33062], 7371423: []})

Need to implemnt: Funk SVD or ALS (Altering Least Square method):

https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1

https://github.com/gbolmier/funk-svd

In [6]:
!pip install git+https://github.com/gbolmier/funk-svd

  Cloning https://github.com/gbolmier/funk-svd to /private/var/folders/qh/pq3w2h0d5md6c1cb16dd9g3m0000gq/T/pip-req-build-a82ashe1
  Created wheel for funk-svd: filename=funk_svd-0.0.1.dev1-py3-none-any.whl size=9044 sha256=7581decccea7893a21457dfbdae8d784b745eea948ee2a01f89d1300079f876d
  Stored in directory: /private/var/folders/qh/pq3w2h0d5md6c1cb16dd9g3m0000gq/T/pip-ephem-wheel-cache-19na1apc/wheels/f8/93/18/db4114b3fafc2eb9a319db1e3b3c3465db51d1fdc1d4f2e769
Successfully built funk-svd
You should consider upgrading via the '/Users/vladimir.nazarov/home/scripts/env/bin/python -m pip install --upgrade pip' command.


In [7]:
from funk_svd.dataset import fetch_ml_ratings
from funk_svd import SVD